In [149]:
from datetime import datetime, timedelta  # Import timedelta from datetime
import pandas as pd

Fin-Bert

In [150]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
model = pipeline("text-classification", model="ProsusAI/finbert")

Data preparation

In [151]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

def scrape_news(company, pages = 1):

    # URL of the website to scrape
    columns = ['datetime', 'title', 'source', 'link']
    df = pd.DataFrame(columns=columns)

    for page in range(1, pages+1):
        url = f'https://markets.businessinsider.com/news/{company}-stock?p={page}'

        
        response = requests.get(url)
        html = response.text
        soup = BeautifulSoup(html, 'lxml')

        articles = soup.find_all('div', class_='latest-news__story')

        for article in articles:
            datetime = article.find('time', class_='latest-news__date').get('datetime')
            title = article.find('a', class_='news-link').text
            source = article.find('span', class_='latest-news__source').text
            link = article.find('a', class_='news-link').get('href')

            df = pd.concat([pd.DataFrame([[datetime, title, source, link]], columns=df.columns), df], ignore_index=True)

    return df
        



In [152]:
# a = soup.find_all('div', class_='latest-news__story')
# len(a)

In [153]:
# df = scrape_news('aapl',150)
df = pd.read_csv('aapl_news.csv')

In [154]:
# save the scraped data to a csv file
# df.to_csv('aapl_news.csv', index=False)

In [155]:
len(df)

7500

In [156]:
# remove time from datetime
df['datetime'] = pd.to_datetime(df['datetime']).dt.date
df.set_index('datetime', inplace=True)

C:\Users\Ishit\AppData\Local\Temp\ipykernel_25028\3893246076.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['datetime'] = pd.to_datetime(df['datetime']).dt.date


In [105]:
# df['sentiment'] = df['title'].apply(lambda x: model(x)[0]['label'])
# df['score'] = df['title'].apply(lambda x: model(x)[0]['score'])

In [106]:
# find missing dates
# missing_dates = pd.date_range(start=df.index.min(), end=df.index.max()).difference(df.index)

In [107]:
# missing_dates

In [108]:
df.source.unique()

array(['InvestorPlace', 'Benzinga', 'Seeking Alpha', 'RTTNews',
       'Markets Insider Automation', 'Premium', 'Business Insider',
       'Moodys', 'PR Newswire', 'TipRanks', 'GlobeNewswire'], dtype=object)

In [109]:
df.index.min(), df.index.max()

(datetime.date(2021, 4, 19), datetime.date(2024, 10, 4))

In [110]:
df.head()

,title,source,link
datetime,,,
2021-04-19,Apple Event Preview: 5 Products With the Poten...,InvestorPlace,/news/stocks/apple-event-preview-5-products-th...
2021-04-19,Midday Market Update: The 10 Most Active Stock...,InvestorPlace,/news/stocks/midday-market-update-the-10-most-...
2021-04-19,Baidu Stock: 3 Reasons To Stay Away,InvestorPlace,/news/stocks/baidu-stock-3-reasons-to-stay-awa...
2021-04-19,Apple's 'Spring Loaded' Event Is Tuesday: Anal...,Benzinga,/news/stocks/apple-s-spring-loaded-event-is-tu...
2021-04-19,"The QQQ, SPY And DIA All Fell Today. Here's Why.",Benzinga,/news/etf/the-qqq-spy-and-dia-all-fell-today-h...


In [111]:
# df.sentiment.value_counts()

In [112]:
# mode_sentiment = df.groupby(df.index)['sentiment'].agg(lambda x: x.mode().tolist())

In [113]:
# mode_sentiment.value_counts()

In [114]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [115]:
import torch
from torch import nn

def classify_sentiment(text_list, batch_size=32):
    all_predictions = []
    all_scores = []

    # Process in batches
    for i in range(0, len(text_list), batch_size):
        pt_batch = tokenizer(
            text_list[i:i + batch_size],  # Get a batch of texts
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        with torch.no_grad():  # Disable gradient calculation
            pt_outputs = model(**pt_batch)
            pt_predictions = nn.functional.softmax(pt_outputs.logits, dim=1)

        max_prediction = torch.argmax(pt_predictions, dim=1).tolist()
        max_scores = torch.max(pt_predictions, dim=1).values.tolist()

        all_predictions.extend(max_prediction)
        all_scores.extend(max_scores)

    return all_predictions, all_scores



In [116]:
x,y = classify_sentiment(["high earnings","low earnings"])

In [117]:
x,y

([2, 2], [0.577485203742981, 0.6063096523284912])

In [131]:
# detached_tensors = [t.detach().item() for t in y]
# detached_tensors

In [133]:
sentiment, scores = classify_sentiment(df['title'].tolist())

In [137]:
df['sentiment'] = sentiment
df['scores']  = [t.detach().item() if isinstance(t, torch.Tensor) else t for t in scores]

# df = pd.read_csv('aapl_news.csv')
# df['sentiment'] = df['title'].apply(lambda x: model(x)[0]['label'])
# df['score'] = df['title'].apply(lambda x: model(x)[0]['score'])
df.head()

,title,source,link,sentiment,scores
datetime,,,,,
2021-04-19,Apple Event Preview: 5 Products With the Poten...,InvestorPlace,/news/stocks/apple-event-preview-5-products-th...,0,0.824311
2021-04-19,Midday Market Update: The 10 Most Active Stock...,InvestorPlace,/news/stocks/midday-market-update-the-10-most-...,2,0.911128
2021-04-19,Baidu Stock: 3 Reasons To Stay Away,InvestorPlace,/news/stocks/baidu-stock-3-reasons-to-stay-awa...,2,0.897794
2021-04-19,Apple's 'Spring Loaded' Event Is Tuesday: Anal...,Benzinga,/news/stocks/apple-s-spring-loaded-event-is-tu...,2,0.899924
2021-04-19,"The QQQ, SPY And DIA All Fell Today. Here's Why.",Benzinga,/news/etf/the-qqq-spy-and-dia-all-fell-today-h...,2,0.566641


In [138]:
# map sentiment to string
df['sentiment'] = df['sentiment'].map({0:'positive', 1:'negative', 2:'neutral'})
df.head()

,title,source,link,sentiment,scores
datetime,,,,,
2021-04-19,Apple Event Preview: 5 Products With the Poten...,InvestorPlace,/news/stocks/apple-event-preview-5-products-th...,positive,0.824311
2021-04-19,Midday Market Update: The 10 Most Active Stock...,InvestorPlace,/news/stocks/midday-market-update-the-10-most-...,neutral,0.911128
2021-04-19,Baidu Stock: 3 Reasons To Stay Away,InvestorPlace,/news/stocks/baidu-stock-3-reasons-to-stay-awa...,neutral,0.897794
2021-04-19,Apple's 'Spring Loaded' Event Is Tuesday: Anal...,Benzinga,/news/stocks/apple-s-spring-loaded-event-is-tu...,neutral,0.899924
2021-04-19,"The QQQ, SPY And DIA All Fell Today. Here's Why.",Benzinga,/news/etf/the-qqq-spy-and-dia-all-fell-today-h...,neutral,0.566641


In [139]:
# save the scraped data to a csv file
df.to_csv('aapl_news_final.csv', index=True)

In [140]:
df.sentiment.value_counts()

sentiment
neutral     4756
positive    1496
negative    1248
Name: count, dtype: int64

In [141]:
def get_sentiment_score(start_date,end_date):
    
    filtered_df = df[(df.index >= start_date) & (df.index <= end_date)]
    
    if filtered_df.empty:
        return 'neutral', 0

    pos_score = filtered_df[ filtered_df['sentiment'] == 'positive']["scores"].sum()
    neg_score = filtered_df[ filtered_df['sentiment'] == 'negative']["scores"].sum()

    pos_score = 0 if pd.isna(pos_score) else pos_score
    neg_score = 0 if pd.isna(neg_score) else neg_score

    mode_sentiment = filtered_df["sentiment"].mode()
    
    # sum based on df.index
    if len(mode_sentiment) == 0:
        return 'neutral', 0
    
    if len(mode_sentiment) > 1:
        return 'positive' if pos_score > neg_score else 'negative', pos_score - neg_score

    return mode_sentiment[0], pos_score - neg_score    



In [142]:
import yfinance as yf

ticker = 'AAPL'
start_date = '2021-04-19'
end_date = '2024-10-05'

stock_data = yf.download(ticker, start=start_date, end=end_date)
stock_data.index = pd.to_datetime(stock_data.index, format='%Y-%m-%d').date

[*********************100%***********************]  1 of 1 completed


In [143]:
stock_data['sentiment'] = None
stock_data['score'] = 0
stock_data.head()

,Open,High,Low,Close,Adj Close,Volume,sentiment,score
2021-04-19,133.509995,135.470001,133.339996,134.839996,132.196259,94264200,None,0
2021-04-20,135.020004,135.529999,131.809998,133.110001,130.500198,94812300,None,0
2021-04-21,132.360001,133.750000,131.300003,133.500000,130.882538,68847100,None,0
2021-04-22,133.039993,134.149994,131.410004,131.940002,129.353119,84566500,None,0
2021-04-23,132.160004,135.119995,132.160004,134.320007,131.686508,78657500,None,0


In [144]:
# stock_data.index.min() == df.index.min()

In [145]:
for i in range(len(stock_data)):
    if i == 0:
        start_date = stock_data.index[i] - timedelta(days=2)
        end_date = stock_data.index[i] - timedelta(days=1)
        
        sentiment, score = get_sentiment_score(start_date, end_date)

        stock_data.iloc[i, stock_data.columns.get_loc('sentiment')] = sentiment
        stock_data.iloc[i, stock_data.columns.get_loc('score')] = score
        
        print(stock_data.iloc[i, stock_data.columns.get_loc('sentiment')], stock_data.iloc[i, stock_data.columns.get_loc('score')])
    
    else:
        start_date = stock_data.index[i-1]
        end_date = stock_data.index[i] - timedelta(days=1)

        sentiment, score = get_sentiment_score(start_date, end_date)

        stock_data.iloc[i, stock_data.columns.get_loc('sentiment')] = sentiment
        stock_data.iloc[i, stock_data.columns.get_loc('score')] = score
        
        print(stock_data.iloc[i, stock_data.columns.get_loc('sentiment')], stock_data.iloc[i, stock_data.columns.get_loc('score')])

neutral 0
neutral 0.8243107199668884
neutral 2.83881276845932
neutral -0.6036558151245117
neutral -0.17318111658096313
neutral 1.1418119072914124
neutral 0.21219390630722046
neutral 0.8919200301170349
neutral 1.4558650851249695
neutral 1.2312824726104736
neutral -0.19358712434768677
neutral -0.5461490750312805
neutral -0.9503692388534546
neutral 0.07993859052658081
neutral 0.0
neutral 0.5392218232154846
positive 1.388765811920166
neutral 0.0
neutral -0.7926970720291138
neutral 0.0
neutral -0.6549042463302612
neutral 0.0
neutral 0.7872363328933716
negative -2.733500063419342
positive 1.0029966831207275
neutral -0.3315157890319824
neutral 0.91475510597229
negative -2.0601555705070496
neutral -0.7537959814071655
neutral 0.0
neutral -1.5462029576301575
negative -0.5013280510902405
neutral 0.0
neutral 0.0
neutral 0.0
neutral -0.027368128299713135
neutral 0.0
neutral -0.9514403343200684
neutral 0.0
neutral 0.0
neutral 0.0
neutral 2.701140820980072
positive 0.9237442016601562
neutral -0.94180

C:\Users\Ishit\AppData\Local\Temp\ipykernel_25028\3558588350.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8243107199668884' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  stock_data.iloc[i, stock_data.columns.get_loc('score')] = score


negative -0.41671568155288696
neutral 0.0
neutral 0.9344689249992371
neutral -0.22331207990646362
neutral 0.8129551410675049
neutral -0.9119824171066284
neutral 0.901074230670929
neutral -0.20275965332984924
neutral 0.6655381321907043
neutral -0.9688603281974792
neutral 0.0
neutral -0.7918971180915833
neutral 1.3915501236915588
neutral 0.0
neutral 0.8089591860771179
neutral 1.692978322505951
neutral 1.4304239749908447
neutral 0.249879390001297
neutral 0.7392380833625793
neutral 0.8640448451042175
neutral -0.9521088004112244
neutral -2.4142554998397827
neutral 0.0
positive 0.039881348609924316
neutral 1.8301353454589844
positive 1.5597469210624695
neutral 0.936924159526825
positive 2.6691693663597107
neutral 0.903657853603363
neutral 0.32556307315826416
neutral 0.5708518028259277
neutral -1.9277640581130981
neutral -0.9191333055496216
neutral -0.12357991933822632
neutral 0.0
negative -0.6295974850654602
neutral 1.248353660106659
neutral -1.9217786192893982
neutral -1.1154568195343018
po

In [146]:
stock_data[:10]

,Open,High,Low,Close,Adj Close,Volume,sentiment,score
2021-04-19,133.509995,135.470001,133.339996,134.839996,132.196259,94264200,neutral,0.000000
2021-04-20,135.020004,135.529999,131.809998,133.110001,130.500198,94812300,neutral,0.824311
2021-04-21,132.360001,133.750000,131.300003,133.500000,130.882538,68847100,neutral,2.838813
2021-04-22,133.039993,134.149994,131.410004,131.940002,129.353119,84566500,neutral,-0.603656
2021-04-23,132.160004,135.119995,132.160004,134.320007,131.686508,78657500,neutral,-0.173181
2021-04-26,134.830002,135.059998,133.559998,134.720001,132.078629,66905100,neutral,1.141812
2021-04-27,135.009995,135.410004,134.110001,134.389999,131.755112,66015800,neutral,0.212194
2021-04-28,134.309998,135.020004,133.080002,133.580002,130.960999,107760100,neutral,0.891920
2021-04-29,136.470001,137.070007,132.449997,133.479996,130.862946,151101000,neutral,1.455865
2021-04-30,131.779999,133.559998,131.070007,131.460007,128.882553,109839500,neutral,1.231282


In [160]:
stock_data.loc[1,'sentiment'] , stock_data.loc[0,'score']

('positive', 0)

In [147]:
stock_data.sentiment.value_counts()

sentiment
neutral     723
positive     80
negative     70
Name: count, dtype: int64

In [148]:
stock_data.to_csv('stock_data_neutral.csv')